# Task 4

> Find the lowest eigenvalue of the following matrix:

> $
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$

> using VQE-like circuits, created by yourself from scratch.


## The approach

Having solved [task 2](QOSF%20-%20Task%202%20%28Bonus%20Question%29.ipynb) we can approach this one in the same way but this time it's much easier because there is only a single parameter. To find the optimum parameter we compare brute force search, evolutionary search and, as a third alternative, gradient descent using a simple **Automatic Differentiation** (AD) loop which is provided by Yao.

## The ansatz

First we prepare the ansatz. The hints suggest using a simple 2-bit circuit with a single parameter, $\theta$. I guess if we don't have this hint we'd need to try various circuits to find a good approximation.

In [1]:
using Yao

ansatz(θ) = chain(2,
                put(1=>H),
                control(1, 2=>X),
                put(1=>Rx(θ)))

ansatz(0)

nqubits: 2
chain
├─ put on (1)
│  └─ H
├─ control(1)
│  └─ (2,) X
└─ put on (1)
   └─ rot(X, 0.0)


## The objective function 

By some trial and error, the matrix, let's call it Q, can be written as
$$2Q = I\otimes I-X\otimes X-Y\otimes Y+Z\otimes Z$$

Using the [same procedure](QOSF%20-%20Task%202%20%28Bonus%20Question%29.ipynb#Measuring-in-the-computational-basis) as we used for task 2, we define the alignment operators for XX, YY and ZZ respectively. The expectation is the sum of the expectations of each term properly aligned in the Z basis.

In [2]:
using StatsBase: mean

XX = repeat(2, Ry(-π/2))
YY = repeat(2, Rx(π/2))
ZZ = repeat(2, I2)

# Convert a measurement {true,false} to Z-operator basis {-1,1}
to_Z_basis(c) = c==0 ? 1 : -1

# Measure a state multiple times and convert 
measure_Z_basis(ψ) = measure(ψ; nshots=100) .|> c -> to_Z_basis(c[1])*to_Z_basis(c[2])

# Calculate the expectation for a particular term, applying appropriate rotation before measurement
expectation_for_term(term_rotation) =
    θ -> zero_state(2) |> ansatz(θ) |> term_rotation |> measure_Z_basis |> mean

expectation(θ) = 
    0.5(1 - expectation_for_term(XX)(θ[1]) - expectation_for_term(YY)(θ[1]) + expectation_for_term(ZZ)(θ[1]))

expectation (generic function with 1 method)

## Brute force search

Since we only have a single parameter, an exhaustive search over the parameter space is possible.

In [3]:
minimum(expectation(θ) for θ in 0:π/32:2π)

-1.0

## Evolutionary search



In [4]:
using Evolutionary
Evolutionary.optimize(
        expectation,
        rand(1)*2π, 
        CMAES(),
        Evolutionary.Options(iterations=200, reltol=0.001))


 * Status: success

 * Candidate solution
    Minimizer:  [3.1105824333247067]
    Minimum:    -1.0
    Iterations: 18

 * Found with
    Algorithm: (15,30)-CMA-ES


# Checking Result

To make sure that the quantum algorithm has found the correct result we compare with the known eigenvalues.

In [5]:
using LinearAlgebra

Q = [1  0  0 0; 
     0  0 -1 0;
     0 -1  0 0; 
     0  0  0 1]

eigen(Q)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
4-element Array{Float64,1}:
 -0.9999999999999989
  1.0
  1.0
  1.0
vectors:
4×4 Array{Float64,2}:
 0.0       1.0   0.0       0.0
 0.707107  0.0  -0.707107  0.0
 0.707107  0.0   0.707107  0.0
 0.0       0.0   0.0       1.0

The minimum eigenvalue is -1.0, and matches the results from the VQE algorithm.

# Autodiff gradient descent

As an alternative to the parameter search methods used above, we can take advantage of the magic of Automatic Differention provided by [YaoExtensions](https://tutorials.yaoquantum.org/dev/generated/quick-start/7.variation-quantum-eigen-solver/) to help perform the gradient descent.

In [6]:
# This only needs to be done once.
#]add YaoExtensions

In [7]:
using Yao.AD, YaoExtensions

circuit = dispatch!(ansatz(rand()*2π), :random)

nqubits: 2
chain
├─ put on (1)
│  └─ H
├─ control(1)
│  └─ (2,) X
└─ put on (1)
   └─ rot(X, 0.8975415358205068)


In [8]:
Q = matblock(Complex{Float64}.(
    [1  0  0 0; 
     0  0 -1 0;
     0 -1  0 0; 
     0  0  0 1]))

learning_rate(t) = exp(1/t)

for t in 1:10
      _, grad = expect'(Q, zero_state(2) => circuit)
      dispatch!(-, circuit, learning_rate(t) * grad)
end
    
println("min = $(real.(expect(Q, zero_state(2)=>circuit)))")

circuit

min = -0.999999999999996


nqubits: 2
chain
├─ put on (1)
│  └─ H
├─ control(1)
│  └─ (2,) X
└─ put on (1)
   └─ rot(X, 3.1415927406490103)


This method has narrowed in on the correct value very quickly. This would be a very powerful way of determining initial values for a more refined search on noisy circuit or simply to prove that a solution exists at all.

# References

**Variational Quantum Eigensolver explained** - Musty Thoughts blog
https://www.mustythoughts.com/variational-quantum-eigensolver-explained

**Yao.jl** - A General Purpose Quantum Computation Simulation Framework
https://docs.yaoquantum.org/stable/

**Evolutionary.jl** Julia evolutionary algorithms package https://wildart.github.io/Evolutionary.jl/stable/